In [ ]:
!nvidia-smi

Thu Sep  8 07:38:12 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P8    13W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!python -m pip install paddlepaddle-gpu==2.3.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 394.0 MB 23 kB/s 
     |████████████████████████████████| 394 kB 60.5 MB/s 


In [ ]:
!pip install --upgrade paddlehub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 212 kB 28.4 MB/s 
     |████████████████████████████████| 1.8 MB 61.4 MB/s 
     |████████████████████████████████| 4.9 MB 58.8 MB/s 
     |████████████████████████████████| 79 kB 7.3 MB/s 
     |████████████████████████████████| 3.0 MB 55.3 MB/s 
     |████████████████████████████████| 1.3 MB 69.6 MB/s 
     |████████████████████████████████| 86 kB 5.6 MB/s 
     |████████████████████████████████| 112 kB 75.4 MB/s 
     |████████████████████████████████| 365 kB 69.4 MB/s 
     |████████████████████████████████| 43 kB 1.9 MB/s 
     |████████████████████████████████| 101 kB 8.9 MB/s 
     |████████████████████████████████| 120 kB 78.1 MB/s 
     |████████████████████████████████| 212 kB 66.9 MB/s 
     |████████████████████████████████| 127 kB 74.3 MB/s 
     |████████████████████████████████| 204 kB 70.9 MB/s 
     |████████████████████████████████

In [ ]:
import os
import pandas as pd
import numpy as np

train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Academic-paper-classification/data/train.csv')
label_list_num=list(train.label_id.unique())
label_list = [str(x) for x in label_list_num]
print(label_list)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35']


In [ ]:
import paddlehub as hub
import paddle
import io, csv
from paddlehub.datasets.base_nlp_dataset import InputExample, TextClassificationDataset

DATA_DIR="/content/drive/MyDrive/Colab Notebooks/Academic-paper-classification/dataset"

class Papers(TextClassificationDataset):
    def __init__(self, tokenizer, mode='train', max_seq_len=128):
        if mode == 'train':
            data_file = 'train_data.csv'
        elif mode == 'dev':
            data_file = 'valid_data.csv'
        super(Papers, self).__init__(
            base_path=DATA_DIR,
            data_file=data_file,
            tokenizer=tokenizer,
            max_seq_len=max_seq_len,
            mode=mode,
            is_file_with_header=True,
            label_list=label_list
            )

    # 解析文本文件里的样本
    def _read_file(self, input_file, is_file_with_header: bool = False):
        if not os.path.exists(input_file):
            raise RuntimeError("The file {} is not found.".format(input_file))
        else:
            with io.open(input_file, "r", encoding="UTF-8") as f:
                reader = csv.reader(f, delimiter="\t")  # ‘\t’分隔数据
                examples = []
                seq_id = 0
                header = next(reader) if is_file_with_header else None
                for line in reader:
                    example = InputExample(guid=seq_id, text_a=line[1], label=line[0])
                    seq_id += 1
                    examples.append(example)
                return examples

model = hub.Module(name="ernie_tiny", task='seq-cls', num_classes=36)
train_dataset = Papers(model.get_tokenizer(), mode='train', max_seq_len=512)
dev_dataset = Papers(model.get_tokenizer(), mode='dev', max_seq_len=512)

# 处理完后查看数据中前2条
for e in train_dataset.examples[:2]:
    print(e)
for e in dev_dataset.examples[:2]:
    print(e)

[2022-09-08 08:39:45,772] [    INFO] - Already cached /root/.paddlenlp/models/ernie-tiny/ernie_tiny.pdparams
[2022-09-08 08:39:47,129] [    INFO] - Already cached /root/.paddlenlp/models/ernie-tiny/vocab.txt
[2022-09-08 08:39:47,131] [    INFO] - Already cached /root/.paddlenlp/models/ernie-tiny/spm_cased_simp_sampled.model
[2022-09-08 08:39:47,135] [    INFO] - Already cached /root/.paddlenlp/models/ernie-tiny/dict.wordseg.pickle
[2022-09-08 08:39:50,898] [    INFO] - tokenizer config file saved in /root/.paddlenlp/models/ernie-tiny/tokenizer_config.json
[2022-09-08 08:39:50,900] [    INFO] - Special tokens file saved in /root/.paddlenlp/models/ernie-tiny/special_tokens_map.json
[2022-09-08 08:39:51,864] [    INFO] - Already cached /root/.paddlenlp/models/ernie-tiny/vocab.txt
[2022-09-08 08:39:51,865] [    INFO] - Already cached /root/.paddlenlp/models/ernie-tiny/spm_cased_simp_sampled.model
[2022-09-08 08:39:51,869] [    INFO] - Already cached /root/.paddlenlp/models/ernie-tiny/dict.

text=一种高空抛物监测方法本发明提供了一种高空抛物监测方法。包括根据建筑物尺寸在建筑物顶部架设摄像头，对建筑物侧面抛物情况进行监控，并将录像文件发送至控制中心；控制中心通过相邻的图片帧对比来判断是否有抛物情况出现，再找到第一次出现抛物目标的图片帧的时间戳并找到所有摄像头在这一时刻拍摄的图像进行二值化处理，得到每个摄像头对应的二值化图像；沿建筑物监控面建立与房号对应的直角坐标系，确定每个摄像头对应的坐标及抛物点和摄像头之间的几何关系，结合二值化图像计算抛物点对应的抛物坐标，再得出抛物坐标对应的房号。本发明通过沿建筑物的监测侧面建立直角坐标系，可根据计算得出的抛物坐标快速定位抛物的房号，便于进行高空抛物追踪、教育和追责。	label=12
text=一种组合式夜景照明装置本实用新型涉及一种组合式夜景照明装置，属于第五立面的照明技术领域，其技术方案要点是包括若干个依次连接的可调支架，可调支架的长度方向上水平设置有至少一列点光源，可调支架的上方安装有罩设点光源的弧形导光板，相邻两个可调支架上的弧形导光板彼此对接，可调支架的两侧安装有线形投光灯，所述线形投光灯朝向可调支架的外侧倾斜向下设置。本实用新型的优点是提供多层次的灯光效果，满足航拍需求及游客的夜景观赏需求。	label=2
text=一种浮箱紧锁装置及水上设备本发明提出一种浮箱紧锁装置及水上设备，所述浮箱紧锁装置用于连接至少两个浮箱箱体，包括：两个连接组件及连接件，两个连接组件沿竖直方向间隔设置于两个浮箱箱体上；所述连接组件包括一对一设于两个浮箱箱体上的两个浮箱紧锁件以及将两个所述浮箱紧锁件锁合到一起并限制两个所述浮箱紧锁件沿水平方向分离的锁合件，所述连接件与两个所述连接组件的锁合件连接，并限制两个所述连接组件的锁合件沿竖直方向分离。本发明通过易拆卸的锁合件来对设置在浮箱箱体上的浮箱紧锁件进行水平方向的连接固定，同时还通过易拆卸的连接件实现对所述锁合件及浮箱紧锁件进行竖直方向上的固定。	label=2
text=压装焊接流水线系统本实用新型涉及加工技术领域，具体涉及一种压装焊接流水线系统，包括机架；分料装置，分料装置安装在机架上且用于对第一工件进行分料；整脚装置，整脚装置安装在机架上且用于对第二工件进行整脚；角度校正装置，角度校正装置安装在机架上且用于对整脚后的第二工件进行角度校正，角度校正装置包括对第二工件的角度

In [ ]:
optimizer = paddle.optimizer.AdamW(learning_rate=2e-5, parameters=model.parameters())
trainer = hub.Trainer(model, optimizer, checkpoint_dir='/content/drive/MyDrive/Colab Notebooks/Academic-paper-classification/cache/checkpoint', use_gpu=True, use_vdl=True)

[2022-09-08 09:07:27,777] [ WARNING] - PaddleHub model checkpoint not found, start from scratch...


In [ ]:
trainer.train(train_dataset, epochs=5, batch_size=32, eval_dataset=dev_dataset, save_interval=1)   # 配置训练参数，启动训练，并指定验证集

[2022-09-08 09:07:41,417] [   TRAIN] - Epoch=1/5, Step=10/24 loss=0.0206 acc=1.0000 lr=0.000020 step/sec=0.87 | ETA 00:02:17
[2022-09-08 09:07:53,145] [   TRAIN] - Epoch=1/5, Step=20/24 loss=0.0203 acc=1.0000 lr=0.000020 step/sec=0.85 | ETA 00:02:18
[2022-09-08 09:08:00,800] [    EVAL] - [Evaluation result] avg_acc=0.5208
[2022-09-08 09:08:05,136] [    EVAL] - Saving best model to /content/drive/MyDrive/Colab Notebooks/Academic-paper-classification/cache/checkpoint/best_model [best acc=0.5208]
[2022-09-08 09:08:05,143] [    INFO] - Saving model checkpoint to /content/drive/MyDrive/Colab Notebooks/Academic-paper-classification/cache/checkpoint/epoch_1
[2022-09-08 09:08:21,406] [   TRAIN] - Epoch=2/5, Step=10/24 loss=0.0128 acc=1.0000 lr=0.000020 step/sec=0.50 | ETA 00:03:01
[2022-09-08 09:08:33,179] [   TRAIN] - Epoch=2/5, Step=20/24 loss=0.0153 acc=1.0000 lr=0.000020 step/sec=0.85 | ETA 00:02:52
[2022-09-08 09:08:40,562] [    EVAL] - [Evaluation result] avg_acc=0.5156
[2022-09-08 09:08

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Academic-paper-classification/data/testA.csv')
test['text_a'] = test['title'] + '。' + test['abstract']
test.head()

,id,title,assignee,abstract,text_a
0,003fd481e65ddc070e38ae05002e16e2,一种耐磨、抗粘钢复合涂层、制备方法及应用,安徽马钢表面技术股份有限公司,本发明公开了一种耐磨、抗粘钢复合涂层、制备方法及应用，包括基体和基体上由内到外依次设置的过渡...,一种耐磨、抗粘钢复合涂层、制备方法及应用。本发明公开了一种耐磨、抗粘钢复合涂层、制备方法及应...
1,549a1cd8228bd10f18395a0625fcc70d,一种用于提高橡胶抗湿滑性的树脂的制备方法及其应用,江苏麒祥高新材料有限公司,本发明公开了一种用于提高橡胶抗湿滑性的树脂的制备方法，第一步：将R树脂和B官能团化合物进行反...,一种用于提高橡胶抗湿滑性的树脂的制备方法及其应用。本发明公开了一种用于提高橡胶抗湿滑性的树脂...
2,f09c4c0332f8966400e06f4def9f1a6d,有机硅改性丙烯酸树脂超亲水防雾涂料及其制作方法,重庆大学,本发明涉及涂料制造领域，本发明公开了一种含有磺酸季铵盐的有机硅改性丙烯酸树脂超亲水低温防雾涂...,有机硅改性丙烯酸树脂超亲水防雾涂料及其制作方法。本发明涉及涂料制造领域，本发明公开了一种含有...
3,06598dd8f3ab092acf2a55dce8be5621,一种空调系统及其控制方法、控制装置,海尔智家股份有限公司,本发明涉及空调领域，公开了一种空调系统，包括室外机和太阳能供热系统，所述太阳能供热系统包括：...,一种空调系统及其控制方法、控制装置。本发明涉及空调领域，公开了一种空调系统，包括室外机和太阳...
4,e70177ba6a54d08abecd80a60fdd9f52,资源申请、分配方法，UE及网络控制单元,中兴通讯股份有限公司,本发明实施例公开了一种资源申请方法及装置，所述方法包括：向网络控制单元发送低时延业务信息；接...,资源申请、分配方法，UE及网络控制单元。本发明实施例公开了一种资源申请方法及装置，所述方法包...


In [ ]:
new = pd.DataFrame(columns=['text'])
new['text'] = test["text_a"]
# 首先将pandas读取的数据转化为array
data_array = np.array(new)
# 然后转化为list形式
data_list =data_array.tolist()

# 定义要进行分类的类别
label_list=list(train.label_id.unique())
label_map = { 
    idx: label_text for idx, label_text in enumerate(label_list)
}

In [ ]:
model = hub.Module(
    name="ernie_tiny",  
    task='seq-cls', 
    load_checkpoint='/content/drive/MyDrive/Colab Notebooks/Academic-paper-classification/cache/checkpoint',
    num_classes=36, 
    label_map=label_map)
predictions = model.predict(data_list, max_seq_len=512, batch_size=2, use_gpu=True)

[2022-09-08 08:13:37,690] [    INFO] - Already cached /root/.paddlenlp/models/ernie-tiny/ernie_tiny.pdparams
[2022-09-08 08:13:42,813] [    INFO] - Loaded parameters from /content/checkpoint/best_model/model.pdparams
[2022-09-08 08:13:42,820] [    INFO] - Already cached /root/.paddlenlp/models/ernie-tiny/vocab.txt
[2022-09-08 08:13:42,822] [    INFO] - Already cached /root/.paddlenlp/models/ernie-tiny/spm_cased_simp_sampled.model
[2022-09-08 08:13:42,825] [    INFO] - Already cached /root/.paddlenlp/models/ernie-tiny/dict.wordseg.pickle
[2022-09-08 08:13:46,342] [    INFO] - tokenizer config file saved in /root/.paddlenlp/models/ernie-tiny/tokenizer_config.json
[2022-09-08 08:13:46,345] [    INFO] - Special tokens file saved in /root/.paddlenlp/models/ernie-tiny/special_tokens_map.json


In [ ]:
test['label'] = predictions
test = test[['id', 'label']]
test.to_csv('/content/drive/MyDrive/Colab Notebooks/Academic-paper-classification/dataset/submission.csv',index=False)